In [1]:
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import csv

In [2]:
DATA_ROOT = '../train_sample'
#DATA_ROOT = '../training'

In [3]:
HITS_SCHEMA = pa.schema({
    'event_id': pa.int32(),
    'hit_id': pa.int32(),
    'x': pa.float32(),
    'y': pa.float32(),
    'z': pa.float32(),
    'volume_id': pa.int8(),
    'layer_id': pa.int8(),
    'module_id': pa.int16(),
})

In [4]:
hits_files = list(Path(DATA_ROOT).glob('event?????????-hits.csv.gz'))

In [5]:
len(hits_files)

50

In [6]:
%%time
tables = [csv.read_csv(f, convert_options=csv.ConvertOptions(column_types=HITS_SCHEMA)) for f in hits_files]

CPU times: user 1.81 s, sys: 923 ms, total: 2.73 s
Wall time: 832 ms


In [7]:
table = pa.concat_tables(tables)

In [8]:
pq.write_table(table, 'test.parquet')

In [9]:
%time pq.read_table('test.parquet')

CPU times: user 195 ms, sys: 61.1 ms, total: 257 ms
Wall time: 220 ms


pyarrow.Table
hit_id: int32
x: float
y: float
z: float
volume_id: int8
layer_id: int8
module_id: int16
----
hit_id: [[1,2,3,4,5,...,24946,24947,24948,24949,24950],[24951,24952,24953,24954,24955,...,54565,54566,54567,54568,54569],...,[122980,122981,122982,122983,122984,...,13123,13124,13125,13126,13127],[13128,13129,13130,13131,13132,...,106221,106222,106223,106224,106225]]
x: [[-71.2343,-74.3945,-82.2111,-71.5644,-51.3558,...,-31.3851,-36.5223,-37.3412,-36.2626,-46.6897],[-47.0361,-46.7315,-45.0943,-49.7965,-44.3724,...,-41.1644,-36.695,-50.3772,-33.8548,-44.7666],...,[-280.974,-215.159,-231.894,-251.618,-175.146,...,-146.79,-145.518,-111.128,-149.553,-107.158],[-136.89,-142.791,-133.311,-126.201,-141.709,...,-929.664,-894.995,-889.412,-928.33,-900.88]]
y: [[-0.966963,-5.98036,-6.09399,-6.81087,-11.9411,...,65.0584,61.3059,60.7077,61.4957,54.1891],[53.8125,54.1437,55.9239,50.811,56.7089,...,-127.302,-113.967,-124.266,-110.056,-107.841],...,[-767.414,-792.9,-785.062,-775.825,-802.351,..

In [10]:
! rm test.parquet